### Импорт необходимых библиотек

 Импортируем библиотеки для работы с данными (`pandas`, `numpy`), сериализации (`json`, `joblib`) и предобработки признаков (`VarianceThreshold`, `MinMaxScaler`) из библиотеки `scikit-learn`.


In [ ]:
import pandas as pd
import numpy as np
import json
import joblib

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from utils import *

### Загрузка обучающих дескрипторов

Загружаем различные типы дескрипторов для обучающей выборки по индексному столбцу `CASRN`:
- ECFP6 в битовом и количественном представлении
- MACCS ключи
- RDKit 2D дескрипторы
- Mordred дескрипторы


In [34]:
train_ecfp6_bits = pd.read_csv('../data/descriptors/train_ecfp6_bits.csv', index_col='CASRN')
train_ecfp6_counts = pd.read_csv('../data/descriptors/train_ecfp6_counts.csv', index_col='CASRN')
train_maccs = pd.read_csv('../data/descriptors/train_maccs.csv', index_col='CASRN')
train_rdkit2d = pd.read_csv('../data/descriptors/train_rdkit2d.csv', index_col='CASRN')
train_mordred = pd.read_csv('../data/descriptors/train_mordred.csv', index_col='CASRN')

### Функция для отбора признаков

Определяем функцию `feature_selection`, которая удаляет признаки с нулевой дисперсией и высоко коррелированные признаки (порог по умолчанию — 0.95). Это помогает уменьшить размерность и избежать мультиколлинеарности.


### Применение отбора признаков к обучающим данным

Применяем функцию `feature_selection` ко всем обучающим дескрипторам и выводим размеры отфильтрованных наборов признаков.


In [37]:
filtered_train_ecfp6_bits = feature_selection(train_ecfp6_bits)
filtered_train_ecfp6_counts = feature_selection(train_ecfp6_counts)
filtered_train_maccs = feature_selection(train_maccs)
filtered_train_rdkit2d = feature_selection(train_rdkit2d)
filtered_train_mordred = feature_selection(train_mordred)

In [38]:
filtered_train_ecfp6_bits.shape, filtered_train_ecfp6_counts.shape,filtered_train_maccs.shape

((8221, 2048), (8221, 1561), (8221, 145))

In [39]:
filtered_train_mordred.shape, filtered_train_rdkit2d.shape

((8221, 462), (8221, 165))

Создаём словарь `dict_features`, в котором для каждого типа дескрипторов сохраняем список имён признаков, прошедших отбор. Сохраняем этот словарь в JSON-файл для последующего использования при обработке тестовых данных.


In [40]:
desc = ['ecfp6_bits', 'ecfp6_counts', 'maccs', 'rdkit2d', 'mordred']
filtered_features = [
    list(filtered_train_ecfp6_bits),
    list(filtered_train_ecfp6_counts),
    list(filtered_train_maccs),
    list(filtered_train_rdkit2d),
    list(filtered_train_mordred),
]

dict_features = dict(zip(desc, filtered_features))

### Сохранение отобранных признаков

In [41]:
with open('../data/descriptors/filtered_features.json', 'w') as f:
    json.dump(dict_features, f)


### Нормализация и сохранение скейлера для RDKit2D

Применяем нормализацию к отфильтрованным RDKit2D дескрипторам и сохраняем обученный `MinMaxScaler` с помощью `joblib`.


In [42]:
norm_train_rdkit2d, scaler_rdkit2d = feature_norm_fit(filtered_train_rdkit2d)

Проверка параметров scaler

In [43]:
scaler_rdkit2d.get_params

<bound method BaseEstimator.get_params of MinMaxScaler()>

Просмотр первой строки нормализованных данных

In [44]:
norm_train_rdkit2d.head(1)

,BalabanJ,BertzCT,Chi0,Chi3v,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,qed
CASRN,,,,,,,,,,,,,,,,,,,,,
23233-88-7,0.286258,0.112142,0.133237,0.157641,0.018166,0.040446,0.0,0.0,0.086767,0.05297,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.382204


Экспорт scaler в файл

In [45]:
joblib.dump(scaler_rdkit2d, '../data/descriptors/scaler_rdkit2d.pkl') 

['../data/descriptors/scaler_rdkit2d.pkl']

Проверка работоспособности scaler 

Загрузка scaler

In [46]:
test_scaler = joblib.load('../data/descriptors/scaler_rdkit2d.pkl')

Тестирование на тренировочных данных

In [47]:
test_rdkit2d = feature_norm_transform(filtered_train_rdkit2d, test_scaler)

Сравнение с оригиналом

In [48]:
test_rdkit2d.head(1) 

,BalabanJ,BertzCT,Chi0,Chi3v,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,qed
CASRN,,,,,,,,,,,,,,,,,,,,,
23233-88-7,0.286258,0.112142,0.133237,0.157641,0.018166,0.040446,0.0,0.0,0.086767,0.05297,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.382204


Обработка Mordred дескрипторов

Нормализация тренировочных данных

Сохранение scaler

In [49]:
norm_train_mordred, scaler_mordred = feature_norm_fit(filtered_train_mordred)
joblib.dump(scaler_mordred, '../data/descriptors/scaler_mordred.pkl')

['../data/descriptors/scaler_mordred.pkl']

In [50]:
test_scaler_mordred = joblib.load('../data/descriptors/scaler_mordred.pkl')

In [51]:
test_mordred = feature_norm_transform(filtered_train_mordred, test_scaler_mordred)
test_mordred.head(1)

,nAcid,nBase,SpAbs_A,SpMax_A,SpMAD_A,LogEE_A,VE1_A,VR1_A,nAromAtom,nAtom,...,JGI6,JGI7,JGI8,JGI9,JGI10,JGT10,SRW03,SRW05,TSRW10,WPath
CASRN,,,,,,,,,,,,,,,,,,,,,
23233-88-7,0.0,0.0,0.13119,0.749405,0.859231,0.622256,0.397997,1.990715e-08,0.25,0.096346,...,0.363075,0.241979,0.338535,0.456364,0.471193,0.540906,0.0,0.0,0.244308,0.004741


In [52]:
norm_train_mordred.head(1)

,nAcid,nBase,SpAbs_A,SpMax_A,SpMAD_A,LogEE_A,VE1_A,VR1_A,nAromAtom,nAtom,...,JGI6,JGI7,JGI8,JGI9,JGI10,JGT10,SRW03,SRW05,TSRW10,WPath
CASRN,,,,,,,,,,,,,,,,,,,,,
23233-88-7,0.0,0.0,0.13119,0.749405,0.859231,0.622256,0.397997,1.990715e-08,0.25,0.096346,...,0.363075,0.241979,0.338535,0.456364,0.471193,0.540906,0.0,0.0,0.244308,0.004741


### Экспорт подготовленных данных

Сохранение тренировочных данных

In [53]:
filtered_train_ecfp6_bits.to_csv('../data/Bmodel_features/modeling_train_ecfp6_bits.csv')
filtered_train_ecfp6_counts.to_csv('../data/Bmodel_features/modeling_train_ecfp6_counts.csv')
filtered_train_maccs.to_csv('../data/Bmodel_features/modeling_train_maccs.csv')
norm_train_rdkit2d.to_csv('../data/Bmodel_features/modeling_train_rdkit2d.csv')
norm_train_mordred.to_csv('../data/Bmodel_features/modeling_train_mordred.csv')

### Подготовка тестовых данных

Функция обработки тестовых данных

In [54]:
def test_feature(df, feature, scaler = None):
    
    with open('../data/descriptors/filtered_features.json') as f:
        dict_features = json.load(f)
        
    if feature not in dict_features.keys():
        raise Exception(f'The feature **{feature}** is not support, please choose from [ecfp6_bits, ecfp6_counts, maccs, rdkit2d, mordred]')
        
    filtered_desc = dict_features[feature]
    df = df[filtered_desc]
    
    if scaler:
        df = feature_norm_transform(df, scaler)
    
    return df

Загрузка тестовых данных

In [55]:
test_ecfp6_bits = pd.read_csv('../data/descriptors/test_ecfp6_bits.csv', index_col='CASRN')
test_ecfp6_counts = pd.read_csv('../data/descriptors/test_ecfp6_counts.csv', index_col='CASRN')
test_maccs = pd.read_csv('../data/descriptors/test_maccs.csv', index_col='CASRN')
test_rdkit2d = pd.read_csv('../data/descriptors/test_rdkit2d.csv', index_col='CASRN')
test_mordred = pd.read_csv('../data/descriptors/test_mordred.csv', index_col='CASRN')

### Финализация тестовых данных

Обработка каждого типа данных

In [56]:
filtered_test_ecfp6_bits = test_feature(test_ecfp6_bits, 'ecfp6_bits')
filtered_test_ecfp6_counts = test_feature(test_ecfp6_counts, 'ecfp6_counts')
filtered_test_maccs = test_feature(test_maccs, 'maccs')

In [57]:
filtered_test_ecfp6_bits.shape, filtered_test_ecfp6_counts.shape, filtered_test_maccs.shape

((2849, 2048), (2849, 1561), (2849, 145))

In [58]:
scaler_mordred = joblib.load('../data/descriptors/scaler_mordred.pkl')
scaler_rdkit2d = joblib.load('../data/descriptors/scaler_rdkit2d.pkl') 

In [59]:
filtered_test_rdkit2d = test_feature(test_rdkit2d, 'rdkit2d', scaler_rdkit2d)

In [60]:
filtered_test_rdkit2d.head(1)

,BalabanJ,BertzCT,Chi0,Chi3v,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,qed
CASRN,,,,,,,,,,,,,,,,,,,,,
130209-82-4,0.207313,0.100235,0.185335,0.22792,0.055726,0.16968,0.0,0.195699,0.129578,0.100139,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012987,0.0,0.254728


In [61]:
filtered_test_mordred = test_feature(test_mordred, 'mordred', scaler_mordred)

In [62]:
filtered_test_mordred.head(1)


,nAcid,nBase,SpAbs_A,SpMax_A,SpMAD_A,LogEE_A,VE1_A,VR1_A,nAromAtom,nAtom,...,JGI6,JGI7,JGI8,JGI9,JGI10,JGT10,SRW03,SRW05,TSRW10,WPath
CASRN,,,,,,,,,,,,,,,,,,,,,
130209-82-4,0.0,0.0,0.186544,0.750389,0.867067,0.682724,0.335865,2.993004e-07,0.125,0.225914,...,0.252108,0.1689,0.266486,0.273758,0.249418,0.412305,0.0,0.437189,0.351957,0.015544


In [63]:
filtered_test_mordred.shape, filtered_test_rdkit2d.shape

((2849, 462), (2849, 165))

Сохранение результатов

In [64]:
filtered_test_ecfp6_bits.to_csv('../data/Bmodel_features/modeling_test_ecfp6_bits.csv')
filtered_test_ecfp6_counts.to_csv('../data/Bmodel_features/modeling_test_ecfp6_counts.csv')
filtered_test_maccs.to_csv('../data/Bmodel_features/modeling_test_maccs.csv')
filtered_test_rdkit2d.to_csv('../data/Bmodel_features/modeling_test_rdkit2d.csv')
filtered_test_mordred.to_csv('../data/Bmodel_features/modeling_test_mordred.csv')